In [ ]:
import datetime
from IPython.display import Markdown as md
import ccxt
import yfinance as yf
import pandas as pd
import plotly
import plotly.express as px
import statsmodels.api as sm
import scipy.optimize as spop

In [ ]:
now = datetime.datetime.now()
md("Last Updated at {}".format(now.strftime("%Y/%m/%d %H:%M")))

In [ ]:
exchange = ccxt.kraken ({"enableRateLimit": True})
candles = exchange.fetch_ohlcv("BTC/USD", "1d")
df_btc = pd.DataFrame(candles)
df_btc = df_btc.iloc[:, [0, 4]]
df_btc.columns = ["time", "btc_usd_close"]
df_btc["time"] = pd.to_datetime(df_btc["time"], unit="ms")
df_btc["str_time"] = pd.to_datetime(df_btc["time"], unit="ms").dt.strftime("%Y/%m/%d")

# Price chart of BTC/USD

In [ ]:
fig = px.line(df_btc, x="time", y="btc_usd_close")
fig.show()

# Price chart of MSTR

In [ ]:
# match start and end date with btc/usd
start_date = list(map(int, df_btc["str_time"].iloc[0].split('/')))
end_date = list(map(int, df_btc["str_time"].iloc[-1].split('/')))
start = datetime.datetime(start_date[0], start_date[1], start_date[2])
end = datetime.datetime(end_date[0], end_date[1], end_date[2])
print(start, end)

# download the stock price 
stock = "MSTR"
df_mstr = yf.download(stock, start=start, end=end, progress=False).reset_index()

In [ ]:
df_mstr = df_mstr[["Date",  "Close"]].rename(columns = {"Date": "time", "Close": "mstr_close"})
df_mstr["str_time"] = df_mstr["time"].dt.strftime("%Y/%m/%d")

In [ ]:
fig = px.line(df_mstr, x="time", y="mstr_close")
fig.show()

# Cointegration test

In [ ]:
df = pd.merge(df_btc, df_mstr, on="str_time", how="inner")